In [20]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import calitp_data_analysis.magics

from rt_analysis import rt_filter_map_plot
import build_speedmaps_index

from IPython.display import display, Markdown
import pandas as pd
import geopandas as gpd
from siuba import *

import shared_utils

In [2]:
catalog = shared_utils.catalog_utils.get_catalog('gtfs_analytics_data')

In [3]:
## parameters cell
itp_id = 4

In [11]:
analysis_date = build_speedmaps_index.ANALYSIS_DATE
import datetime as dt
# if itp_id == 293:
#     analysis_date = dt.date(2023, 9, 13)

In [12]:
speedmap_index = pd.read_parquet(f'_rt_progress_{analysis_date}.parquet') >> filter(_.organization_itp_id == itp_id)
# speedmap_index = pd.read_parquet(f'_rt_progress_{analysis_date}.parquet') >> filter(_.organization_name.str.contains('Torr'))

In [17]:
speedmap_index

,organization_name,organization_itp_id,organization_source_record_id,caltrans_district,_is_current,analysis_date,schedule_gtfs_dataset_key,status
96,Alameda-Contra Costa Transit District,4,recOZgevYf7Jimm9L,04 - Oakland,True,2024-10-16,c499f905e33929a641f083dad55c521e,speedmap_segs_available


In [19]:
path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.shape_stop_single_segment_detail}_{analysis_date}.parquet'

In [21]:
speedmap_segs = gpd.read_parquet(path) #  aggregated

In [22]:
speedmap_segs.columns

Index(['schedule_gtfs_dataset_key', 'shape_array_key', 'shape_id', 'route_id',
       'route_short_name', 'stop_pair', 'stop_pair_name', 'segment_id',
       'time_of_day', 'p50_mph', 'n_trips', 'p20_mph', 'p80_mph', 'name',
       'caltrans_district', 'organization_source_record_id',
       'organization_name', 'base64_url', 'geometry'],
      dtype='object')

In [23]:
speedmap_segs = speedmap_segs >> filter(_.schedule_gtfs_dataset_key == speedmap_index.schedule_gtfs_dataset_key.iloc[0])

In [24]:
speedmap_segs >> head(3)

,schedule_gtfs_dataset_key,shape_array_key,shape_id,route_id,route_short_name,stop_pair,stop_pair_name,segment_id,time_of_day,p50_mph,n_trips,p20_mph,p80_mph,name,caltrans_district,organization_source_record_id,organization_name,base64_url,geometry
526507,c499f905e33929a641f083dad55c521e,f59dce41f6c4e6cf0b9dbfc1a29095f1,shp-7-53,7,7,53019__53372,San Pablo Av & Wall Av__San Pablo Av & Conlon Av,53019-53372-1,AM Peak,5.80,1,5.80,5.80,Bay Area 511 AC Transit Schedule,04 - Oakland,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,"LINESTRING (-122.32022 37.92809, -122.32036 37..."
526508,c499f905e33929a641f083dad55c521e,f59dce41f6c4e6cf0b9dbfc1a29095f1,shp-7-53,7,7,53019__53372,San Pablo Av & Wall Av__San Pablo Av & Conlon Av,53019-53372-1,Early AM,22.14,1,22.14,22.14,Bay Area 511 AC Transit Schedule,04 - Oakland,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,"LINESTRING (-122.32022 37.92809, -122.32036 37..."
526509,c499f905e33929a641f083dad55c521e,f59dce41f6c4e6cf0b9dbfc1a29095f1,shp-7-53,7,7,53019__53372,San Pablo Av & Wall Av__San Pablo Av & Conlon Av,53019-53372-1,Evening,22.18,4,16.06,23.19,Bay Area 511 AC Transit Schedule,04 - Oakland,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,"LINESTRING (-122.32022 37.92809, -122.32036 37..."


In [25]:
speedmap_segs.time_of_day.unique()

array(['AM Peak', 'Early AM', 'Evening', 'Midday', 'PM Peak', 'Owl'],
      dtype=object)

In [26]:
peak_test = speedmap_segs >> filter(_.time_of_day == 'AM Peak')

In [27]:
time_periods = [(x, shared_utils.rt_utils.categorize_time_of_day(x)) for x in range(1, 25)]

In [28]:
time_periods

[(1, 'Owl'),
 (2, 'Owl'),
 (3, 'Owl'),
 (4, 'Early AM'),
 (5, 'Early AM'),
 (6, 'Early AM'),
 (7, 'AM Peak'),
 (8, 'AM Peak'),
 (9, 'AM Peak'),
 (10, 'Midday'),
 (11, 'Midday'),
 (12, 'Midday'),
 (13, 'Midday'),
 (14, 'Midday'),
 (15, 'PM Peak'),
 (16, 'PM Peak'),
 (17, 'PM Peak'),
 (18, 'PM Peak'),
 (19, 'PM Peak'),
 (20, 'Evening'),
 (21, 'Evening'),
 (22, 'Evening'),
 (23, 'Evening'),
 (24, 'Evening')]

In [31]:
am_peak_hrs = len([x[1] for x in time_periods if 'AM Peak' in x[1]])

In [32]:
am_peak_hrs

3

In [33]:
def prep_gdf(gdf, hrs):
    
    gdf = gdf.to_crs(calitp_data_analysis.geography_utils.CA_NAD83Albers)
    gdf['trips_per_hour'] = gdf.n_trips / hrs

    ## shift to right side of road to display direction
    gdf.geometry = gdf.geometry.apply(shared_utils.rt_utils.try_parallel)
    gdf = gdf.apply(shared_utils.rt_utils.arrowize_by_frequency, axis=1)

    gdf = gdf >> arrange(_.trips_per_hour)

    return gdf

In [34]:
peak_test = prep_gdf(peak_test, am_peak_hrs)

In [60]:
cmap = shared_utils.rt_utils.ACCESS_ZERO_THIRTY_COLORSCALE
url = shared_utils.rt_utils.ACCESS_SPEEDMAP_LEGEND_URL

##  adding identifiers for map display

In [36]:
peak_test >> head(2)

,schedule_gtfs_dataset_key,shape_array_key,shape_id,route_id,route_short_name,stop_pair,stop_pair_name,segment_id,time_of_day,p50_mph,n_trips,p20_mph,p80_mph,name,caltrans_district,organization_source_record_id,organization_name,base64_url,geometry,trips_per_hour
526507,c499f905e33929a641f083dad55c521e,f59dce41f6c4e6cf0b9dbfc1a29095f1,shp-7-53,7,7,53019__53372,San Pablo Av & Wall Av__San Pablo Av & Conlon Av,53019-53372-1,AM Peak,5.80,1,5.80,5.80,Bay Area 511 AC Transit Schedule,04 - Oakland,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,"POLYGON ((-203712.125 -7192.575, -203734.516 -...",0.333333
527527,c499f905e33929a641f083dad55c521e,261a73dd824eab6a8fb3616a320d00e0,shp-62-01,62,62,50553__57505,23rd Av & Foothill Blvd (Garfield School)__23r...,50553-57505-1,AM Peak,13.48,1,13.48,13.48,Bay Area 511 AC Transit Schedule,04 - Oakland,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,"POLYGON ((-196613.695 -23524.639, -196614.785 ...",0.333333


##  to-do

* where does new n_trips come from? should be schedule-based for speedmaps applications...
* split am/pm peak
* https://github.com/cal-itp/data-analyses/issues/1218 upstream add route identifiers

In [37]:
peak_test.columns

Index(['schedule_gtfs_dataset_key', 'shape_array_key', 'shape_id', 'route_id',
       'route_short_name', 'stop_pair', 'stop_pair_name', 'segment_id',
       'time_of_day', 'p50_mph', 'n_trips', 'p20_mph', 'p80_mph', 'name',
       'caltrans_district', 'organization_source_record_id',
       'organization_name', 'base64_url', 'geometry', 'trips_per_hour'],
      dtype='object')

## export map

In [38]:
speedmap_state = shared_utils.rt_utils.set_state_export(
                                    peak_test, filename=f'{itp_id}_new', map_type='speedmap',
                                    color_col='p20_mph', cmap=cmap, legend_url=url,
                                    cache_seconds=0, map_title=f'Speedmap Segs {analysis_date}')

writing to calitp-map-tiles/testing/4_new.geojson.gz


In [39]:
speedmap_state

{'state_dict': {'name': 'null',
  'layers': [{'name': 'Speedmap Segs 2024-10-16',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/4_new.geojson.gz',
    'properties': {'stroked': False,
     'highlight_saturation_multiplier': 0.5,
     'tooltip_speed_key': 'p20_mph'},
    'type': 'speedmap'}],
  'lat_lon': (37.777276879932664, -122.20458759054884),
  'zoom': 13,
  'legend_url': 'https://storage.googleapis.com/calitp-map-tiles/speeds_legend.svg'},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJTcGVlZG1hcCBTZWdzIDIwMjQtMTAtMTYiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3Rlc3RpbmcvNF9uZXcuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41LCAidG9vbHRpcF9zcGVlZF9rZXkiOiAicDIwX21waCJ9LCAidHlwZSI6ICJzcGVlZG1hcCJ9XSwgImxhdF9sb24iOiBbMzcuNzc3Mjc2ODc5OTMyNjY0LCAtMTIyLjIwNDU4NzU5MDU0ODg0XSwgInpvb20iOiAxMywgImxlZ2VuZF91cmwiO

## other test

In [46]:
time_of_day = 'PM Peak'

In [47]:
period_speeds = speedmap_segs >> filter(_.time_of_day == time_of_day)

In [48]:
hrs = len([x[1] for x in time_periods if x[1] == time_of_day])

In [49]:
hrs

5

In [50]:
period_test = prep_gdf(period_speeds, hrs)

In [57]:
time_of_day_lower = time_of_day.lower().replace(' ', '_')

In [61]:
speedmap_state = shared_utils.rt_utils.set_state_export(
                                    period_test, filename=f'{itp_id}_{time_of_day_lower}_new', map_type='speedmap',
                                    color_col='p20_mph', cmap=cmap, legend_url=url,
                                    cache_seconds=0, map_title=f'Speedmap Segs {time_of_day} {analysis_date}')

writing to calitp-map-tiles/testing/4_pm_peak_new.geojson.gz


In [62]:
speedmap_state

{'state_dict': {'name': 'null',
  'layers': [{'name': 'Speedmap Segs PM Peak 2024-10-16',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/4_pm_peak_new.geojson.gz',
    'properties': {'stroked': False,
     'highlight_saturation_multiplier': 0.5,
     'tooltip_speed_key': 'p20_mph'},
    'type': 'speedmap'}],
  'lat_lon': (37.771147425687175, -122.19794783624761),
  'zoom': 13,
  'legend_url': 'https://storage.googleapis.com/calitp-map-tiles/speeds_legend_color_access.svg'},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJTcGVlZG1hcCBTZWdzIFBNIFBlYWsgMjAyNC0xMC0xNiIsICJ1cmwiOiAiaHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2NhbGl0cC1tYXAtdGlsZXMvdGVzdGluZy80X3BtX3BlYWtfbmV3Lmdlb2pzb24uZ3oiLCAicHJvcGVydGllcyI6IHsic3Ryb2tlZCI6IGZhbHNlLCAiaGlnaGxpZ2h0X3NhdHVyYXRpb25fbXVsdGlwbGllciI6IDAuNSwgInRvb2x0aXBfc3BlZWRfa2V5IjogInAyMF9tcGgifSwgInR5cGUiOiAic3BlZWRtYXAifV0sICJsYXRfbG9uIjogWzM3Ljc3MTE0NzQyNTY4NzE3NSwgLTEyMi4xOTc5

## other test

In [63]:
time_of_day = 'Evening'

In [64]:
period_speeds = speedmap_segs >> filter(_.time_of_day == time_of_day)

In [65]:
hrs = len([x[1] for x in time_periods if x[1] == time_of_day])

In [66]:
hrs

5

In [67]:
period_test = prep_gdf(period_speeds, hrs)

In [68]:
time_of_day_lower = time_of_day.lower().replace(' ', '_')

In [69]:
speedmap_state = shared_utils.rt_utils.set_state_export(
                                    period_test, filename=f'{itp_id}_{time_of_day_lower}_new', map_type='speedmap',
                                    color_col='p20_mph', cmap=cmap, legend_url=url,
                                    cache_seconds=0, map_title=f'Speedmap Segs {time_of_day} {analysis_date}')

writing to calitp-map-tiles/testing/4_evening_new.geojson.gz


In [70]:
speedmap_state

{'state_dict': {'name': 'null',
  'layers': [{'name': 'Speedmap Segs Evening 2024-10-16',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/4_evening_new.geojson.gz',
    'properties': {'stroked': False,
     'highlight_saturation_multiplier': 0.5,
     'tooltip_speed_key': 'p20_mph'},
    'type': 'speedmap'}],
  'lat_lon': (37.771823438071266, -122.20351640174141),
  'zoom': 13,
  'legend_url': 'https://storage.googleapis.com/calitp-map-tiles/speeds_legend_color_access.svg'},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJTcGVlZG1hcCBTZWdzIEV2ZW5pbmcgMjAyNC0xMC0xNiIsICJ1cmwiOiAiaHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2NhbGl0cC1tYXAtdGlsZXMvdGVzdGluZy80X2V2ZW5pbmdfbmV3Lmdlb2pzb24uZ3oiLCAicHJvcGVydGllcyI6IHsic3Ryb2tlZCI6IGZhbHNlLCAiaGlnaGxpZ2h0X3NhdHVyYXRpb25fbXVsdGlwbGllciI6IDAuNSwgInRvb2x0aXBfc3BlZWRfa2V5IjogInAyMF9tcGgifSwgInR5cGUiOiAic3BlZWRtYXAifV0sICJsYXRfbG9uIjogWzM3Ljc3MTgyMzQzODA3MTI2NiwgLTEyMi4yMDM1

## other test

In [71]:
time_of_day = 'Owl'

In [72]:
period_speeds = speedmap_segs >> filter(_.time_of_day == time_of_day)

In [73]:
hrs = len([x[1] for x in time_periods if x[1] == time_of_day])

In [74]:
hrs

3

In [75]:
period_test = prep_gdf(period_speeds, hrs)

In [76]:
time_of_day_lower = time_of_day.lower().replace(' ', '_')

In [77]:
speedmap_state = shared_utils.rt_utils.set_state_export(
                                    period_test, filename=f'{itp_id}_{time_of_day_lower}_new', map_type='speedmap',
                                    color_col='p20_mph', cmap=cmap, legend_url=url,
                                    cache_seconds=0, map_title=f'Speedmap Segs {time_of_day} {analysis_date}')

writing to calitp-map-tiles/testing/4_owl_new.geojson.gz


In [78]:
speedmap_state

{'state_dict': {'name': 'null',
  'layers': [{'name': 'Speedmap Segs Owl 2024-10-16',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/4_owl_new.geojson.gz',
    'properties': {'stroked': False,
     'highlight_saturation_multiplier': 0.5,
     'tooltip_speed_key': 'p20_mph'},
    'type': 'speedmap'}],
  'lat_lon': (37.7948400139263, -122.23030873967325),
  'zoom': 13,
  'legend_url': 'https://storage.googleapis.com/calitp-map-tiles/speeds_legend_color_access.svg'},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJTcGVlZG1hcCBTZWdzIE93bCAyMDI0LTEwLTE2IiwgInVybCI6ICJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY2FsaXRwLW1hcC10aWxlcy90ZXN0aW5nLzRfb3dsX25ldy5nZW9qc29uLmd6IiwgInByb3BlcnRpZXMiOiB7InN0cm9rZWQiOiBmYWxzZSwgImhpZ2hsaWdodF9zYXR1cmF0aW9uX211bHRpcGxpZXIiOiAwLjUsICJ0b29sdGlwX3NwZWVkX2tleSI6ICJwMjBfbXBoIn0sICJ0eXBlIjogInNwZWVkbWFwIn1dLCAibGF0X2xvbiI6IFszNy43OTQ4NDAwMTM5MjYzLCAtMTIyLjIzMDMwODczOTY3MzI1XSwgInpvb2